In [1]:
import pandas as pd 
import numpy as np
import h5py
import tiledb 
import random 
import pyBigWig
random.seed(1234)

In [18]:
attribute_of_interest='fc_bigwig'
batch_size=1000
vector_length=1000
task="ENCSR000EID"
chromsizes_file="hg38.chrom.sizes"

In [19]:
#get chromsizes dictionary 
chromsizes=pd.read_csv(chromsizes_file,sep='\t',header=None,index_col=0)[1].to_dict()
chromsizes
chroms=list(chromsizes.keys())
print(chroms)

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']


In [20]:
## random batch of data -- "batch_size" of genomic regions of "vector_length" each 
regions=[] 
for batch_entry in range(batch_size): 
    cur_chrom=random.choice(chroms)
    cur_start=random.randint(vector_length,chromsizes[cur_chrom]-vector_length)
    regions.append([cur_chrom,cur_start,cur_start+vector_length])
print(regions[0:10])

[['chr21', 5881111, 5882111], ['chr5', 58260538, 58261538], ['chr2', 116172450, 116173450], ['chr13', 98882536, 98883536], ['chr11', 55599083, 55600083], ['chr22', 25582971, 25583971], ['chr9', 81748786, 81749786], ['chr14', 42810213, 42811213], ['chr22', 38773097, 38774097], ['chr5', 142980389, 142981389]]


In [21]:
## convert from "bed file" coordinates to tiledb indices (tiledb stores bases 0 - 3 billion in one vector)

#get starting genomewide position for each chrom 
chrom_to_start={} 
cur_start=0 
for chrom in chroms: 
    chrom_to_start[chrom]=cur_start 
    cur_start+=chromsizes[chrom]
print(chrom_to_start)

#convert region "bed" coordinates to tdb indices
regions_tdb=[] 
for region in regions: 
    tdb_index=chrom_to_start[region[0]]+region[1]
    regions_tdb.append(tdb_index)


{'chr1': 0, 'chr2': 248956422, 'chr3': 491149951, 'chr4': 689445510, 'chr5': 879660065, 'chr6': 1061198324, 'chr7': 1232004303, 'chr8': 1391350276, 'chr9': 1536488912, 'chr10': 1674883629, 'chr11': 1808681051, 'chr12': 1943767673, 'chr13': 2077042982, 'chr14': 2191407310, 'chr15': 2298451028, 'chr16': 2400442217, 'chr17': 2490780562, 'chr18': 2574038003, 'chr19': 2654411288, 'chr20': 2713028904, 'chr21': 2777473071, 'chr22': 2824183054, 'chrX': 2875001522, 'chrY': 3031042417}


In [22]:
tdb_indices=[slice(i,i+vector_length-1) for i in regions_tdb]


3088269832

In [23]:
## Tiledb Test 1: open tiledb array 
tdb_array=tiledb.open('.'.join([task,'tdb']),'r',ctx=tiledb.Ctx())

In [24]:
##Tiledb Test 2: extract values for a batch of data 
batch_tdb=tdb_array.query(attrs=[attribute_of_interest]).multi_index[tdb_indices][attribute_of_interest]
batch_tdb=np.reshape(batch_tdb,(batch_size,-1))


KeyboardInterrupt: 

In [ ]:
## HDF5 Test 1: # open hdf5 file for reading 
hdf5_local=h5py.File('.'.join([task,'hdf5']),mode='r')

In [10]:
## HDF5 Test 2: read regions for task ENCSR000EID from a local hdf5 file 
batch_hdf5=np.full((batch_size,vector_length),np.nan)
region_index=0
for region in regions:
    batch_hdf5[region_index,:]=hdf5_local[region[0]][region[1]:region[2]]
    region_index+=1

In [11]:
## pyBigWig  Test 1: open BigWig for reading 
bigwig_local=pyBigWig.open("ENCSR000EID.merged.nodup.fc.signal.bigwig",'r')

In [12]:
## pyBigWig  Test 2: read regions for task ENCSR000EID from a local BigWig
batch_bw=np.full((batch_size,vector_length),np.nan)
region_index=0
for region in regions:
    batch_bw[region_index,:]=bigwig_local.values(region[0],region[1],region[2],numpy=True)
    region_index+=1


In [13]:
#pyBigWig treats empty regions as NaN, convert to 0 
batch_bw=np.nan_to_num(batch_bw)

In [14]:
## numpy Test 1: extract chr1 from a bigwig and save it
## IF WANT TO LIMIT TO CHROM 1 

#bigwig_local=pyBigWig.open("ENCSR000EID.merged.nodup.fc.signal.bigwig",'r')
#signal = np.nan_to_num(bigwig_local.values('chr1', 0, -1, numpy=True))
#np.save("ENCSR000EID.chr1.npy", signal)

##genomewide signal extraction -- this takes 35 minutes, so recommended to use pre-generated file on mitra: 
## http://mitra.stanford.edu/kundaje/annashch/query_speed_test/ENCSR000EID.npy 
##genomewide signal from tdb for storing in numpy 

#signal=tdb_array.query(attrs=[attribute_of_interest]).multi_index[0:3088269832][attribute_of_interest]
#np.save("ENCSR000EID.npy", signal)

In [17]:
# numpy Test 2: read regions for the task from a memory mapped numpy array

numpy_local = np.load("ENCSR000EID.npy", mmap_mode='r')

batch_npy=np.full((batch_size,vector_length),np.nan)
region_index=0
for region in regions_tdb:
    batch_npy[region_index]=numpy_local[region:region+vector_length,0]
    region_index+=1

In [16]:
# numpy Test 3: read regions 
signal=np.load("ENCSR000EID.npy")


NameError: name 'signal' is not defined

In [ ]:
batch_npy2=np.full((batch_size,vector_length),np.nan)
region_index=0
for region in regions_tdb:
    batch_npy2[region_index]=signal[region:region+vector_length,0]
    region_index+=1

In [ ]:
batch_tdb.sum(), batch_hdf5.sum(), batch_bw.sum(), batch_npy.sum(), batch_npy2.sum()

In [ ]:
#make sure we're getting the same batch each time
assert sum(sum(batch_tdb==batch_hdf5))==batch_size*vector_length
assert sum(sum(batch_tdb==batch_bw))==batch_size*vector_length
assert sum(sum(batch_hdf5==batch_bw))==batch_size*vector_length
assert sum(sum(batch_npy==batch_tdb))==batch_size*vector_length
assert sum(sum(batch_npy2==batch_tdb))==batch_size*vector_length
print("all assertions met")

In [ ]:
type(batch_tdb[0][0])

In [ ]:
type(batch_hdf5[0][0])

In [ ]:
type(batch_bw[0][0])

In [ ]:
type(batch_npy[0][0])

In [ ]:
type(batch_npy2[0][0])